In [1]:
import pandas as pd
import glob
import regex as re
import querygen as qg

In [2]:
files = glob.glob("results/*.csv")
dfs = [pd.read_csv(f, index_col=0) for f in files]
files

['results/eval_results_UniProt_oneshot_NousResearch-Hermes-3-Llama-3.2-3B-GGUF.csv',
 'results/eval_results_DBpedia_oneshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_BTO_oneshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_UniProt_oneshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_DNB_oneshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_BTO_oneshot_NousResearch-Hermes-3-Llama-3.2-3B-GGUF.csv',
 'results/eval_results_DNB_oneshot_NousResearch-Hermes-3-Llama-3.2-3B-GGUF.csv',
 'results/eval_results_DBpedia_oneshot_NousResearch-Hermes-3-Llama-3.2-3B-GGUF.csv']

In [3]:

def graph_from_erl(erl: qg.EnrichedEntitiesRelations):
    G = qg.nx.DiGraph()
    for node in erl.entities:
        G.add_node(node.identifier, label=node.type)
    for link in erl.relations:
        G.add_edge(
            link.entity,
            link.target,
            weight=link.link.instance_count,
            label=link.relation,
        )
    return G

In [4]:
for f, df in zip(files, dfs):
    # data=re.findall(r"results/eval_results_(\w+)_.*", f)[0]
    # prompt_type=re.findall(r"results/eval_results_(\w+)_.*", f)[0]
    model = re.findall(r"results/eval_results_\w+_(.+)\.csv", f)[0]
    # df['prompt_type'] = prompt_type
    df["model"] = (
        model.replace("_", " ")
        .replace("-", " ")
        .rstrip("-GGUF")
        .lstrip("NousResearch Hermes 3 Llama")
        .lstrip(".2")
        .lstrip(".3")
    )

df = pd.concat(dfs)
df

,f1_score,precision,recall,ged,normed_ged,response,model,cfg_name,zeroshot,stage,erl,generator,n_nodes,seed,erl_loaded
0,0.800000,1.000000,0.666667,1.0,0.800000,"a Citation has a published in with Journal, an...",3B,UniProt,False,unconstrained,"{""relations"":[{""entity"":""Citation"",""relation"":...",templated,3,2,"relations=[EnrichedRelation(entity='Citation',..."
1,0.800000,1.000000,0.666667,5.0,0.285714,"a Citation has a published in with Journal, an...",3B,UniProt,False,final,"{""relations"":[{""entity"":""Citation"",""relation"":...",templated,3,2,"relations=[EnrichedRelation(entity='Citation',..."
2,0.500000,1.000000,0.333333,4.0,0.200000,a Reviewed protein (records) has a domain with...,3B,UniProt,False,unconstrained,"{""relations"":[{""entity"":""Reviewed protein (rec...",templated,3,3,relations=[EnrichedRelation(entity='Reviewed p...
3,0.400000,0.500000,0.333333,10.0,0.090909,a Reviewed protein (records) has a domain with...,3B,UniProt,False,final,"{""relations"":[{""entity"":""Reviewed protein (rec...",templated,3,3,relations=[EnrichedRelation(entity='Reviewed p...
4,0.666667,0.666667,0.666667,2.0,0.600000,a Simple Sequence has a member of with Cluster...,3B,UniProt,False,unconstrained,"{""relations"":[{""entity"":""Simple Sequence"",""rel...",templated,3,5,relations=[EnrichedRelation(entity='Simple Seq...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2591,1.000000,1.000000,1.000000,9.0,0.500000,a ethnic group has a population place with pop...,3B,DBpedia,False,final,"{""relations"":[{""entity"":""ethnic group"",""relati...",templated,7,297,relations=[EnrichedRelation(entity='ethnic gro...
2592,0.833333,1.000000,0.714286,9.0,0.307692,"A person was born in a populated place, curren...",3B,DBpedia,False,unconstrained,"{""relations"":[{""entity"":""person"",""relation"":""b...",llama,7,299,"relations=[EnrichedRelation(entity='person', r..."
2593,0.727273,1.000000,0.571429,7.0,0.461538,"A person was born in a populated place, curren...",3B,DBpedia,False,final,"{""relations"":[{""entity"":""person"",""relation"":""b...",llama,7,299,"relations=[EnrichedRelation(entity='person', r..."
2594,0.769231,0.833333,0.714286,20.0,0.047619,a person has a birth place with populated plac...,3B,DBpedia,False,unconstrained,"{""relations"":[{""entity"":""person"",""relation"":""b...",templated,7,299,"relations=[EnrichedRelation(entity='person', r..."


In [5]:
df

,f1_score,precision,recall,ged,normed_ged,response,model,cfg_name,zeroshot,stage,erl,generator,n_nodes,seed,erl_loaded
0,0.800000,1.000000,0.666667,1.0,0.800000,"a Citation has a published in with Journal, an...",3B,UniProt,False,unconstrained,"{""relations"":[{""entity"":""Citation"",""relation"":...",templated,3,2,"relations=[EnrichedRelation(entity='Citation',..."
1,0.800000,1.000000,0.666667,5.0,0.285714,"a Citation has a published in with Journal, an...",3B,UniProt,False,final,"{""relations"":[{""entity"":""Citation"",""relation"":...",templated,3,2,"relations=[EnrichedRelation(entity='Citation',..."
2,0.500000,1.000000,0.333333,4.0,0.200000,a Reviewed protein (records) has a domain with...,3B,UniProt,False,unconstrained,"{""relations"":[{""entity"":""Reviewed protein (rec...",templated,3,3,relations=[EnrichedRelation(entity='Reviewed p...
3,0.400000,0.500000,0.333333,10.0,0.090909,a Reviewed protein (records) has a domain with...,3B,UniProt,False,final,"{""relations"":[{""entity"":""Reviewed protein (rec...",templated,3,3,relations=[EnrichedRelation(entity='Reviewed p...
4,0.666667,0.666667,0.666667,2.0,0.600000,a Simple Sequence has a member of with Cluster...,3B,UniProt,False,unconstrained,"{""relations"":[{""entity"":""Simple Sequence"",""rel...",templated,3,5,relations=[EnrichedRelation(entity='Simple Seq...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2591,1.000000,1.000000,1.000000,9.0,0.500000,a ethnic group has a population place with pop...,3B,DBpedia,False,final,"{""relations"":[{""entity"":""ethnic group"",""relati...",templated,7,297,relations=[EnrichedRelation(entity='ethnic gro...
2592,0.833333,1.000000,0.714286,9.0,0.307692,"A person was born in a populated place, curren...",3B,DBpedia,False,unconstrained,"{""relations"":[{""entity"":""person"",""relation"":""b...",llama,7,299,"relations=[EnrichedRelation(entity='person', r..."
2593,0.727273,1.000000,0.571429,7.0,0.461538,"A person was born in a populated place, curren...",3B,DBpedia,False,final,"{""relations"":[{""entity"":""person"",""relation"":""b...",llama,7,299,"relations=[EnrichedRelation(entity='person', r..."
2594,0.769231,0.833333,0.714286,20.0,0.047619,a person has a birth place with populated plac...,3B,DBpedia,False,unconstrained,"{""relations"":[{""entity"":""person"",""relation"":""b...",templated,7,299,"relations=[EnrichedRelation(entity='person', r..."


In [6]:
df["prompt_type"] = df["zeroshot"].apply(lambda x: "zero shot" if x else "one shot")
df.drop(columns=["zeroshot"], inplace=True)
df

,f1_score,precision,recall,ged,normed_ged,response,model,cfg_name,stage,erl,generator,n_nodes,seed,erl_loaded,prompt_type
0,0.800000,1.000000,0.666667,1.0,0.800000,"a Citation has a published in with Journal, an...",3B,UniProt,unconstrained,"{""relations"":[{""entity"":""Citation"",""relation"":...",templated,3,2,"relations=[EnrichedRelation(entity='Citation',...",one shot
1,0.800000,1.000000,0.666667,5.0,0.285714,"a Citation has a published in with Journal, an...",3B,UniProt,final,"{""relations"":[{""entity"":""Citation"",""relation"":...",templated,3,2,"relations=[EnrichedRelation(entity='Citation',...",one shot
2,0.500000,1.000000,0.333333,4.0,0.200000,a Reviewed protein (records) has a domain with...,3B,UniProt,unconstrained,"{""relations"":[{""entity"":""Reviewed protein (rec...",templated,3,3,relations=[EnrichedRelation(entity='Reviewed p...,one shot
3,0.400000,0.500000,0.333333,10.0,0.090909,a Reviewed protein (records) has a domain with...,3B,UniProt,final,"{""relations"":[{""entity"":""Reviewed protein (rec...",templated,3,3,relations=[EnrichedRelation(entity='Reviewed p...,one shot
4,0.666667,0.666667,0.666667,2.0,0.600000,a Simple Sequence has a member of with Cluster...,3B,UniProt,unconstrained,"{""relations"":[{""entity"":""Simple Sequence"",""rel...",templated,3,5,relations=[EnrichedRelation(entity='Simple Seq...,one shot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2591,1.000000,1.000000,1.000000,9.0,0.500000,a ethnic group has a population place with pop...,3B,DBpedia,final,"{""relations"":[{""entity"":""ethnic group"",""relati...",templated,7,297,relations=[EnrichedRelation(entity='ethnic gro...,one shot
2592,0.833333,1.000000,0.714286,9.0,0.307692,"A person was born in a populated place, curren...",3B,DBpedia,unconstrained,"{""relations"":[{""entity"":""person"",""relation"":""b...",llama,7,299,"relations=[EnrichedRelation(entity='person', r...",one shot
2593,0.727273,1.000000,0.571429,7.0,0.461538,"A person was born in a populated place, curren...",3B,DBpedia,final,"{""relations"":[{""entity"":""person"",""relation"":""b...",llama,7,299,"relations=[EnrichedRelation(entity='person', r...",one shot
2594,0.769231,0.833333,0.714286,20.0,0.047619,a person has a birth place with populated plac...,3B,DBpedia,unconstrained,"{""relations"":[{""entity"":""person"",""relation"":""b...",templated,7,299,"relations=[EnrichedRelation(entity='person', r...",one shot


In [7]:
df_clean = df.dropna()
df_clean.describe()

,f1_score,precision,recall,ged,normed_ged,n_nodes,seed
count,9768.000000,9768.000000,9768.000000,9768.000000,9768.000000,9768.000000,9768.000000
mean,0.795393,0.839769,0.775562,3.988227,0.535197,4.956593,148.233006
std,0.234100,0.228692,0.251207,4.523169,0.323981,1.640133,85.551849
min,0.153846,0.166667,0.142857,0.000000,-0.307692,3.000000,0.000000
25%,0.666667,0.666667,0.500000,1.000000,0.250000,3.000000,75.000000
50%,0.888889,1.000000,0.857143,3.000000,0.500000,5.000000,147.000000
75%,1.000000,1.000000,1.000000,6.000000,0.800000,7.000000,222.000000
max,1.000000,1.000000,1.000000,77.000000,1.000000,10.000000,299.000000


In [8]:
df["erl_loaded"] = df["erl"].apply(qg.EnrichedEntitiesRelations.model_validate_json)
# df["response_loaded"] = df["response"].apply(lambda x: qg.EnrichedEntitiesRelations.model_validate_json(x) if isinstance(x, str) else None)
df["erl_graph"] = df["erl_loaded"].apply(graph_from_erl)
# df["response_graph"] = df["response"].apply(lambda x: graph_from_erl if x is not None else None)

In [9]:
# df["normed_ged"] = 1 - df["ged"] / df["erl_graph"].apply(
#     lambda x: (len(x.nodes) + len(x.nodes))
# )

In [10]:
means = df_clean.groupby(
    ["n_nodes", "prompt_type", "stage", "model", "generator", "cfg_name"]
)[["f1_score", "normed_ged"]].mean()
counts = df_clean.groupby(
    ["n_nodes", "stage", "prompt_type", "model", "generator", "cfg_name"]
)[["f1_score", "normed_ged"]].count()
means

f1_score  \
n_nodes prompt_type stage         model generator cfg_name             
3       one shot    final          3B   human     DBpedia   0.688889   
                                        llama     BTO       0.792958   
                                                  DBpedia   0.779446   
                                                  DNB       0.500000   
                                                  UniProt   0.776923   
...                                                              ...   
7       one shot    unconstrained 1 8B  templated UniProt   0.768842   
10      one shot    final          3B   human     DBpedia   0.650000   
                                  1 8B  human     DBpedia   0.726852   
                    unconstrained  3B   human     DBpedia   0.652219   
                                  1 8B  human     DBpedia   0.576190   

                                                            normed_ged  
n_nodes prompt_type stage         model generator cfg_name              
3       one shot    final          3B   human     DBpedia     0.511111  
                                        llama     BTO         0.437514  
                                                  DBpedia     0.484292  
                                                  DNB         0.409091  
                                                  UniProt     0.516667  
...                                                                ...  
7       one shot    unconstrained 1 8B  templated UniProt     0.507594  
10      one shot    final          3B   human     DBpedia     0.133089  
                                  1 8B  human     DBpedia     0.269799  
                    unconstrained  3B   human     DBpedia     0.170426  
                                  1 8B  human     DBpedia     0.177945  

[108 rows x 2 columns]

In [11]:
counts

f1_score  \
n_nodes stage         prompt_type model generator cfg_name             
3       final         one shot     3B   human     DBpedia          3   
                                        llama     BTO             71   
                                                  DBpedia        234   
                                                  DNB             11   
                                                  UniProt         26   
...                                                              ...   
7       unconstrained one shot    1 8B  templated UniProt         94   
10      final         one shot     3B   human     DBpedia          3   
                                  1 8B  human     DBpedia          3   
        unconstrained one shot     3B   human     DBpedia          3   
                                  1 8B  human     DBpedia          3   

                                                            normed_ged  
n_nodes stage         prompt_type model generator cfg_name              
3       final         one shot     3B   human     DBpedia            3  
                                        llama     BTO               71  
                                                  DBpedia          234  
                                                  DNB               11  
                                                  UniProt           26  
...                                                                ...  
7       unconstrained one shot    1 8B  templated UniProt           94  
10      final         one shot     3B   human     DBpedia            3  
                                  1 8B  human     DBpedia            3  
        unconstrained one shot     3B   human     DBpedia            3  
                                  1 8B  human     DBpedia            3  

[108 rows x 2 columns]

In [12]:
column_map = {
    "f1_score": "$F_1$",
    "normed_ged": "$GED_{s}$",
    "stage": "Stage",
    "prompt_type": "Prompt Type",
    "model": "Version \\& Size",
    "n_nodes": "$k$",
    "generator": "Query Origin",
}

## Compare Query Origins on DBpedia with 8B model

In [13]:
query_gen_compare = df_clean[
    df_clean["model"].str.contains("8B")
    & df_clean["cfg_name"].str.contains("DBpedia")
    & df_clean["prompt_type"].str.contains("one shot")
]
grouped_query_gen_compare = query_gen_compare.groupby(
    ["n_nodes", "generator", "stage"]
)[["f1_score", "normed_ged"]].mean()
grouped_query_gen_compare = grouped_query_gen_compare.rename(
    columns=column_map
).rename_axis([column_map["n_nodes"], column_map["generator"], column_map["stage"]], axis=0)
with open("tables/compare_gen_query.tex", "w") as f:
    f.write(
        grouped_query_gen_compare.to_latex(
            float_format="%.2f",
            caption="Comparison of query generation methods for LLama 3.1 8B on DBpedia with one shot prompts.",
            label="tab:compare_gen_query",
        )
    )
grouped_query_gen_compare

$F_1$  $GED_{s}$
$k$ Query Origin Stage                             
3   human        final          0.450000   0.400000
                 unconstrained  0.450000   0.300000
    llama        final          0.765532   0.536743
                 unconstrained  0.708833   0.346873
    templated    final          0.876903   0.744060
                 unconstrained  0.838685   0.552417
5   human        final          0.833333   0.277273
                 unconstrained  0.700000   0.216667
    llama        final          0.736856   0.417864
                 unconstrained  0.653362   0.244521
    templated    final          0.828778   0.671006
                 unconstrained  0.789124   0.488500
7   llama        final          0.676559   0.337288
                 unconstrained  0.611406   0.177877
    templated    final          0.852057   0.652216
                 unconstrained  0.803481   0.478953
10  human        final          0.726852   0.269799
                 unconstrained  0.576190   0.177945

## Compare models across Datasets

In [14]:
datasets = ["DBpedia", "UniProt", "BTO"]
grouped_dfs = {}
for dataset in datasets:
    model_compare = df_clean[df_clean["cfg_name"].str.strip() == dataset]
    grouped_model_compare = model_compare.groupby(
        ["n_nodes", "stage", "prompt_type", "model"]
    )[["f1_score", "normed_ged"]].mean()
    grouped_model_compare = grouped_model_compare.rename(
        columns=column_map
    ).rename_axis(
        [
            column_map["n_nodes"],
            column_map["stage"],
            column_map["prompt_type"],
            column_map["model"],
        ],
        axis=0,
    )
    grouped_dfs[dataset] = grouped_model_compare
    with open(f"tables/compare_model_{dataset}.tex", "w") as f:
        f.write(
            grouped_model_compare.to_latex(
                float_format="%.2f",
                caption=f"Comparison of models for {dataset}",
                label=f"tab:compare_model_{dataset}",
                # sparsify=False
            )
        )
merged_grouped = pd.merge(
    grouped_dfs["DBpedia"],
    grouped_dfs["UniProt"],
    on=["$k$", "Stage", "Prompt Type", "Version \\& Size"],
    suffixes=tuple([f"_{dataset}" for dataset in datasets[:2]]),
)
merged_grouped = pd.merge(
    merged_grouped,
    grouped_dfs["BTO"],
    on=["$k$", "Stage", "Prompt Type", "Version \\& Size"],
    suffixes=("", f"_{datasets[2]}"),
)
rename_map = {f"$F_1$_{dataset}": f"{dataset} $F_1$" for dataset in datasets} | {
    f"$GED_{{s}}$_{dataset}": f"{dataset} $GED_{{s}}$" for dataset in datasets
}
merged_grouped.rename(
    columns=rename_map,
    inplace=True,
)
with open("tables/compare_model_merged.tex", "w") as f:
    f.write(
        merged_grouped.to_latex(
            float_format="%.2f",
            caption="Comparison of models for DBpedia and UniProt",
            label="tab:compare_model_merged",
        )
    )

In [17]:
merged_grouped

DBpedia $F_1$  \
$k$ Stage         Prompt Type Version \& Size                  
3   final         one shot     3B                   0.829236   
                              1 8B                  0.822261   
    unconstrained one shot     3B                   0.798872   
                              1 8B                  0.775509   
5   final         one shot     3B                   0.765024   
                              1 8B                  0.785963   
    unconstrained one shot     3B                   0.741473   
                              1 8B                  0.725424   
7   final         one shot     3B                   0.725173   
                              1 8B                  0.766328   
    unconstrained one shot     3B                   0.735760   
                              1 8B                  0.709654   

                                               DBpedia $GED_{s}$  \
$k$ Stage         Prompt Type Version \& Size                      
3   final         one shot     3B                       0.559775   
                              1 8B                      0.644548   
    unconstrained one shot     3B                       0.493906   
                              1 8B                      0.454190   
5   final         one shot     3B                       0.472629   
                              1 8B                      0.551126   
    unconstrained one shot     3B                       0.390946   
                              1 8B                      0.373481   
7   final         one shot     3B                       0.425913   
                              1 8B                      0.498377   
    unconstrained one shot     3B                       0.372337   
                              1 8B                      0.331880   

                                               UniProt $F_1$  \
$k$ Stage         Prompt Type Version \& Size                  
3   final         one shot     3B                   0.890731   
                              1 8B                  0.892648   
    unconstrained one shot     3B                   0.789116   
                              1 8B                  0.749687   
5   final         one shot     3B                   0.870229   
                              1 8B                  0.877408   
    unconstrained one shot     3B                   0.807667   
                              1 8B                  0.734394   
7   final         one shot     3B                   0.860765   
                              1 8B                  0.879787   
    unconstrained one shot     3B                   0.780820   
                              1 8B                  0.746668   

                                               UniProt $GED_{s}$     $F_1$  \
$k$ Stage         Prompt Type Version \& Size                                
3   final         one shot     3B                       0.634027  0.921891   
                              1 8B                      0.809844  0.909707   
    unconstrained one shot     3B                       0.521460  0.886212   
                              1 8B                      0.469876  0.634066   
5   final         one shot     3B                       0.612836  0.933099   
                              1 8B                      0.826817  0.910292   
    unconstrained one shot     3B                       0.533089  0.884578   
                              1 8B                      0.483260  0.616850   
7   final         one shot     3B                       0.603516  0.930799   
                              1 8B                      0.848339  0.870365   
    unconstrained one shot     3B                       0.504934  0.882311   
                              1 8B                      0.484620  0.591895   

                                               $GED_{s}$  
$k$ Stage         Prompt Type Version \& Size             
3   final         one shot     3B               0.532217  
                              

In [15]:
dataset

'BTO'

In [16]:
df_clean["cfg_name"].str.strip() == "DBpedia"

0       False
1       False
2       False
3       False
4       False
        ...  
2591     True
2592     True
2593     True
2594     True
2595     True
Name: cfg_name, Length: 9768, dtype: bool